<a href="https://colab.research.google.com/github/Infant-Joshva/AI_Agents-Learning_Materials/blob/main/Practical_Part/Pinecone_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U pinecone sentence-transformers

In [2]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec

In [ ]:
#setup

PINECONE-API_KEY = ""
index_name = "pinecone-demo"